# Install face-alignment package
Environment: Google colab

In [ ]:
!pip install --upgrade torch

In [ ]:
%mkdir face-alignment
%cd face-alignment
!git init .
!git remote add origin https://github.com/1adrianb/face-alignment.git
!git fetch origin
!git checkout master

%cd ../


#!apt update
#!apt install -y cmake

**Install dlib (CUDA enabled or CPU version)**

CUDA enabled dlib

dlib w/o CUDA

# Generate binary masks

```bash
    Inputs:
        Images from ./faceA and ./faceB
    Outputs:
        Eyes binary masks, which are saved to ./binary_masks/faceA_eyes and ./binary_masks/faceB_eyes respectively
```

In [ ]:
from glob import glob
from pathlib import PurePath, Path
import cv2
import numpy as np
import matplotlib as plt
import face_alignment

In [ ]:

img_dir_src = './face_src/rgb'  # source face
img_dir_dst = './face_dst/rgb'  # target face
img_dir_src_bm_eyes = "./face_src/binary_mask/faceA_eyes"
img_dir_dst_bm_eyes = "./face_dst/binary_mask/faceB_eyes"


In [ ]:
fns_face_src = glob(f"{img_dir_src}/*.*")
fns_face_dst = glob(f"{img_dir_dst}/*.*")

In [ ]:
fa = face_alignment.FaceAlignment(face_alignment.LandmarksType._2D, enable_cuda=True, flip_input=False)

In [ ]:
Path(f"binary_mask/faceA_eyes").mkdir(parents=True, exist_ok=True)
Path(f"binary_mask/faceB_eyes").mkdir(parents=True, exist_ok=True)

In [ ]:
fns_face_not_detected = []

for idx, fns in enumerate([fns_face_src, fns_face_dst]):
    if idx == 0:
        save_path = img_dir_src_bm_eyes
    elif idx == 1:
        save_path = img_dir_dst_bm_eyes

        # create binary mask for each training image
    for fn in fns:
        raw_fn = PurePath(fn).parts[-1]

        x = plt.imread(fn)
        x = cv2.resize(x, (256, 256))
        preds = fa.get_landmarks(x)

        if preds is not None:
            preds = preds[0]
            mask = np.zeros_like(x)

            # Draw right eye binary mask
            pnts_right = [(preds[i, 0], preds[i, 1]) for i in range(36, 42)]
            hull = cv2.convexHull(np.array(pnts_right)).astype(np.int32)
            mask = cv2.drawContours(mask, [hull], 0, (255, 255, 255), -1)

            # Draw left eye binary mask
            pnts_left = [(preds[i, 0], preds[i, 1]) for i in range(42, 48)]
            hull = cv2.convexHull(np.array(pnts_left)).astype(np.int32)
            mask = cv2.drawContours(mask, [hull], 0, (255, 255, 255), -1)

            # Draw mouth binary mask
            #pnts_mouth = [(preds[i,0],preds[i,1]) for i in range(48,60)]
            #hull = cv2.convexHull(np.array(pnts_mouth)).astype(np.int32)
            #mask = cv2.drawContours(mask,[hull],0,(255,255,255),-1)

            mask = cv2.dilate(mask, np.ones((13, 13), np.uint8), iterations=1)
            mask = cv2.GaussianBlur(mask, (7, 7), 0)

        else:
            mask = np.zeros_like(x)
            print(f"No faces were detected in image '{fn}''")
            fns_face_not_detected.append(fn)

        plt.imsave(fname=f"{save_path}/{raw_fn}", arr=mask, format="jpg")

In [ ]:
num_faceA = len(glob(img_dir_src + "/*.*"))
num_faceB = len(glob(img_dir_dst + "/*.*"))

print("Nuber of processed images: " + str(num_faceA + num_faceB))
print("Number of image(s) with no face detected: " + str(len(fns_face_not_detected)))

# Randomly diaplay a face image and its result binary mask

In [ ]:
face = np.random.choice(["A", "B"])

dir_face = img_dir_src if face == "A" else img_dir_dst
fns_face = fns_face_src if face == "A" else fns_face_dst
num_face = len(glob(dir_face + "/*.*"))
rand_idx = np.random.randint(num_face)
rand_fn = fns_face[rand_idx]
raw_fn = PurePath(rand_fn).parts[-1]
mask_fn = f"{img_dir_src_bm_eyes}/{raw_fn}" if face == "A" else f"{img_dir_dst_bm_eyes}/{raw_fn}"
im = plt.imread(rand_fn)
mask = plt.imread(mask_fn)

if rand_fn in fns_face_not_detected:
    print("========== На этом изображении не было обнаружено никаких лиц! ==========")

fig = plt.figure(figsize=(15, 6))
plt.subplot(1, 3, 1)
plt.grid('off')
plt.imshow(im)
plt.subplot(1, 3, 2)
plt.grid('off')
plt.imshow(mask)
plt.subplot(1, 3, 3)
plt.grid('off')
plt.imshow((mask / 255 * im).astype(np.uint8))

#fa.get_landmarks(x)

# Случайно отображать изображения, на котором не обнаружено лица.

In [ ]:
num_no_face_img = len(fns_face_not_detected)
rand_idx = np.random.randint(num_no_face_img)
x = plt.imread(fns_face_not_detected[rand_idx])
#x = cv2.resize(x, (256,256))

plt.grid('off')
plt.imshow(x)

#fa.get_landmarks(x)